## Data Wrangling with Python: Intro to Pandas
Note: Notebook adapted from [here](https://github.com/EricElmoznino/lighthouse_pandas_tutorial/blob/master/pandas_tutorial.ipynb) & [here](https://github.com/sedv8808/LighthouseLabs/tree/main/W02D2) & from LHL's [21 Day Data Challenge](https://data-challenge.lighthouselabs.ca/start)
#### Instructor: Simon Dawkins
**Agenda:**
 - Why Pandas?
 - Pandas Basics
     - Pandas Series vs. Pandas DataFrames
     - .loc() vs. iloc()
 - Pandas Advance
     - Filtering
     - Group-bys
 - Pandas Exercises
     - Challenge 1
     - Challenge 2

### Pandas: Why Pandas? What is it? 


To do data anlaysis with Python, Pandas is a great tool to for dealing with data in a tabular and time series formats. Designed by Wes McKinney as an attempt to port R's dataframes to python. 

- Python Package for working with **tables**
- Similar to SQL & Excel
    - Faster sometimes - more optimized for non-distributed platforms
    - More features to manipulate, transform, and aggregate data
- Easy to handle messy and missing data
- Great at working with large datasets
- When combing with other Python libraries, it's fairly easy to create beautiful and customazied visuals. Easy integration with Matplotlib, Seaborn, Plotly.
- Easy integration with machine learning plugins (sckit-learn, etc)
    
    
-----------
To read more about, Wes McKinney, the creator of Pandas, check out the article below.

1. https://qz.com/1126615/the-story-of-the-most-important-tool-in-data-science/

--------------


## Think of how we would try to represent a table in Python?


In [1]:
#A dicitonary of lists example
students = {
    'student_id': [1, 2, 3, 4,5,6],
    'name': ['Daenerys', 'Jon', 'Arya', 'Sansa', 'Eddard', 'Khal Drogo'],
    'course_mark': [82, 100, 12, 76, 46, 20],
    'species': ['cat', 'human', 'cat', 'human', 'human', 'human']
}

**What are some operations we might want to do on this data?**

- 1. Select a subset of columns
- 2. Filter out some rows based on an attribute
- 3. Group by some attribute
- 4. Compute some aggregate values within groups
- 5. Save to a file

How about we try out one of these to see how easy it is

### Return a table with the mean course mark per-species with vanilla Python

In [2]:
# Return a table with the mean course mark per species

species_sums = {} # Tables of Sums
species_counts = {} # Count per Species
for i in range(len(students['species'])):  # iterating over the rows
    species = students['species'][i] # every row number I get species 
    course_mark = students['course_mark'][i] #  and course mark
    if species not in species_sums: # Intializing Species if not in list
        species_sums[species] = 0
        species_counts[species] = 0
    species_sums[species] += course_mark # Add each course mark for each species
    species_counts[species] += 1 

species_means = {}
                                  
for species in species_sums: # for every unique species we found
    species_means[species] = species_sums[species] / species_counts[species] #sum/count

species_means

{'cat': 47.0, 'human': 60.5}

- Did you like looking at this? 
- Does this look fun to do? 
- what about with 10000 rows and 50 columns?
- Super Tiring.

## Pandas Version

In [3]:
import pandas as pd

# Can take in a dictionry of list to instantiate a DataFrame
students = pd.DataFrame(students)
students

,student_id,name,course_mark,species
0,1,Daenerys,82,cat
1,2,Jon,100,human
2,3,Arya,12,cat
3,4,Sansa,76,human
4,5,Eddard,46,human
5,6,Khal Drogo,20,human


In [4]:
species_means = students[['species', 'course_mark']].groupby('species').mean()
species_means = students.groupby('species')['course_mark'].mean()
species_means

species
cat      47.0
human    60.5
Name: course_mark, dtype: float64

### Dissecting the above code:


In [5]:
#Step 1: Filter out the columns we want to keep
students_filtered = students[['species','course_mark']]
students_filtered

,species,course_mark
0,cat,82
1,human,100
2,cat,12
3,human,76
4,human,46
5,human,20


In [6]:
# Step 2: Group by species column
students_grouped_by_species = students_filtered.groupby('species') 
students_grouped_by_species

In [7]:
#Step 3: Specify how to aggregate the course-mark column
species_means = students_grouped_by_species.mean()

In [8]:
species_means

,course_mark
species,
cat,47.0
human,60.5


#### As shown, Pandas makes use of vectorized operations. 


- Rather than use for-loops, we specify the operation that will apply to the structure as a whole (i.e. all the rows)
- By vectorizing, **the code becomes more concise and more readable**
- Pandas is optimized for vectorized operations (parallel vs. serial computation), which makes them **much faster**
- It is almost always possible to vectorize operations on Pandas data types - series have a single data type


### Getting Started: Pandas Series & Pandas DataFrames

There are two Pandas data types of interest:

- Series (column)
    - A pandas series is similar to an array but it has an index. The index is constant, and doesnt change through the operations we apply to the series. 
- DataFrame (table)
    - A pandas dataframe is an object that is similar to a collection of pandas series.

In [9]:
# One way to construct a Series
grades = pd.Series([82, 100, 12, 76, 46, 20]) 
grades

0     82
1    100
2     12
3     76
4     46
5     20
dtype: int64

In [10]:
#We can specify some index when building a series. 
grades = pd.Series([82, 100, 12, 76, 46, 20], 
                   index = ['Daenerys', 'Jon', 'Arya', 'Sansa', 'Eddard', 'Khal Drogo'] ) 

grades

Daenerys       82
Jon           100
Arya           12
Sansa          76
Eddard         46
Khal Drogo     20
dtype: int64

In [11]:
print("The values:", grades.values)
print("The indexes:", grades.index)

The values: [ 82 100  12  76  46  20]
The indexes: Index(['Daenerys', 'Jon', 'Arya', 'Sansa', 'Eddard', 'Khal Drogo'], dtype='object')


**Note:** The underlying index is still 0, 1, 2, 3.... and we can still index on that:

In [12]:
grades[2]

12

### Pandas DataFrames

In [13]:
# One way to construct a DataFrame
df = pd.DataFrame({
    'name': ['Daenerys', 'Jon', 'Arya', 'Sansa'],
    'course_mark': [82, 100, 12, 76],
    'species': ['human', 'human', 'cat', 'human']},
    index=[1412, 94, 9351, 14])
df

,name,course_mark,species
1412,Daenerys,82,human
94,Jon,100,human
9351,Arya,12,cat
14,Sansa,76,human


#### Reading a CSV file

We'll use the function `read_csv()` to load the data into our notebook

- The `read_csv()` function can read data from a locally saved file or from a URL
- We'll store the data as a variable `df_pokemon`

In [14]:
pokemon = pd.read_csv('pokemon.csv')

In [15]:
pokemon

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


**What do we see here?**
- Each row of the table is an observation, containing data of a single pokemon

In [16]:
pokemon.shape

(800, 13)

For large DataFrames, it's often useful to display just the first few or last few rows:

In [17]:
pokemon.head

<bound method NDFrame.head of        #                   Name   Type 1  Type 2  Total  HP  Attack  Defense  \
0      1              Bulbasaur    Grass  Poison    318  45      49       49   
1      2                Ivysaur    Grass  Poison    405  60      62       63   
2      3               Venusaur    Grass  Poison    525  80      82       83   
3      3  VenusaurMega Venusaur    Grass  Poison    625  80     100      123   
4      4             Charmander     Fire     NaN    309  39      52       43   
..   ...                    ...      ...     ...    ...  ..     ...      ...   
795  719                Diancie     Rock   Fairy    600  50     100      150   
796  719    DiancieMega Diancie     Rock   Fairy    700  50     160      110   
797  720    HoopaHoopa Confined  Psychic   Ghost    600  80     110       60   
798  720     HoopaHoopa Unbound  Psychic    Dark    680  80     160       60   
799  721              Volcanion     Fire   Water    600  80     110      120   

     Sp. 

In [18]:
pokemon.tail

<bound method NDFrame.tail of        #                   Name   Type 1  Type 2  Total  HP  Attack  Defense  \
0      1              Bulbasaur    Grass  Poison    318  45      49       49   
1      2                Ivysaur    Grass  Poison    405  60      62       63   
2      3               Venusaur    Grass  Poison    525  80      82       83   
3      3  VenusaurMega Venusaur    Grass  Poison    625  80     100      123   
4      4             Charmander     Fire     NaN    309  39      52       43   
..   ...                    ...      ...     ...    ...  ..     ...      ...   
795  719                Diancie     Rock   Fairy    600  50     100      150   
796  719    DiancieMega Diancie     Rock   Fairy    700  50     160      110   
797  720    HoopaHoopa Confined  Psychic   Ghost    600  80     110       60   
798  720     HoopaHoopa Unbound  Psychic    Dark    680  80     160       60   
799  721              Volcanion     Fire   Water    600  80     110      120   

     Sp. 

## Data at a Glance

`pandas` provides many ways to quickly and easily summarize your data:
- How many rows and columns are there?
- What are all the column names and what type of data is in each column?
- How many values are missing in each column or row?
- Numerical data: What is the average and range of the values?
- Text data: What are the unique values and how often does each occur?

### Peeking into the pokemon dataset

- Similar with getting familar with SQL tables, it is often a great idea to look at the pandas dataframes we are working with. Below are some of the basic methods to glance at a dataset. 

In [19]:
# Getting the Columns
pokemon.columns

Index(['#', 'Name', 'Type 1', 'Type 2', 'Total', 'HP', 'Attack', 'Defense',
       'Sp. Atk', 'Sp. Def', 'Speed', 'Generation', 'Legendary'],
      dtype='object')

In [20]:
# Getting Summary Statistics
pokemon.describe

<bound method NDFrame.describe of        #                   Name   Type 1  Type 2  Total  HP  Attack  Defense  \
0      1              Bulbasaur    Grass  Poison    318  45      49       49   
1      2                Ivysaur    Grass  Poison    405  60      62       63   
2      3               Venusaur    Grass  Poison    525  80      82       83   
3      3  VenusaurMega Venusaur    Grass  Poison    625  80     100      123   
4      4             Charmander     Fire     NaN    309  39      52       43   
..   ...                    ...      ...     ...    ...  ..     ...      ...   
795  719                Diancie     Rock   Fairy    600  50     100      150   
796  719    DiancieMega Diancie     Rock   Fairy    700  50     160      110   
797  720    HoopaHoopa Confined  Psychic   Ghost    600  80     110       60   
798  720     HoopaHoopa Unbound  Psychic    Dark    680  80     160       60   
799  721              Volcanion     Fire   Water    600  80     110      120   

     

In [21]:
# accessing columns with dot
pokemon['Name']


0                  Bulbasaur
1                    Ivysaur
2                   Venusaur
3      VenusaurMega Venusaur
4                 Charmander
               ...          
795                  Diancie
796      DiancieMega Diancie
797      HoopaHoopa Confined
798       HoopaHoopa Unbound
799                Volcanion
Name: Name, Length: 800, dtype: object

In [44]:
# Checking for Missing Data
pokemon.isnull().sum()

#               0
Name            0
Type 1          0
Type 2        386
Total           0
HP              0
Attack          0
Defense         0
Sp. Atk         0
Sp. Def         0
Speed           0
Generation      0
Legendary       0
dtype: int64

In [45]:
# getting info
pokemon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           800 non-null    int64 
 1   Name        800 non-null    object
 2   Type 1      800 non-null    object
 3   Type 2      414 non-null    object
 4   Total       800 non-null    int64 
 5   HP          800 non-null    int64 
 6   Attack      800 non-null    int64 
 7   Defense     800 non-null    int64 
 8   Sp. Atk     800 non-null    int64 
 9   Sp. Def     800 non-null    int64 
 10  Speed       800 non-null    int64 
 11  Generation  800 non-null    int64 
 12  Legendary   800 non-null    bool  
dtypes: bool(1), int64(9), object(3)
memory usage: 75.9+ KB


## The .loc() vs .iloc() method


To select rows and columns at the same time, we use the syntax `.loc[ROWS SLICE, COLUMNS SLICE]`:

In [46]:
# simple slice
# Notice the square brackets on loc
pokemon.loc[20:30:5, 'Attack']

20    45
25    81
30    55
Name: Attack, dtype: int64

In [47]:
# Getting more than one column
pokemon.loc[[1,10],['Attack','Name', 'Legendary']]

,Attack,Name,Legendary
1,62,Ivysaur,False
10,63,Wartortle,False


In [48]:
# pass a list for the rows
pokemon.loc[10:18,"Name": "Attack"]

,Name,Type 1,Type 2,Total,HP,Attack
10,Wartortle,Water,NaN,405,59,63
11,Blastoise,Water,NaN,530,79,83
12,BlastoiseMega Blastoise,Water,NaN,630,79,103
13,Caterpie,Bug,NaN,195,45,30
14,Metapod,Bug,NaN,205,50,20
15,Butterfree,Bug,Flying,395,60,45
16,Weedle,Bug,Poison,195,40,35
17,Kakuna,Bug,Poison,205,45,25
18,Beedrill,Bug,Poison,395,65,90


In [49]:
# pass a range range of column values
p]

SyntaxError: unmatched ']' (2361801911.py, line 2)

In [50]:
#iloc is used for integer based indexing



### Modifying a Column or Creating a new column

In [51]:
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [52]:
# Make sure make a copy of your dataframe before doing any modification
# This is generaly best practice, and it will be easier for you to keep track of the various
# changes and states of your dataframe. 
pokemon2 = pokemon.copy()

In [53]:
# create a new column
# Combine Attack + Special Attack
pokemon2["new column called total attack"] = pokemon2["Attack"] + pokemon2["Sp. Atk"]
pokemon2["new column called total attack"]

0      114
1      142
2      182
3      222
4      112
      ... 
795    200
796    320
797    260
798    330
799    240
Name: new column called total attack, Length: 800, dtype: int64

In [54]:
# modify existing column
pokemon2['Total'] = pokemon2['Total']  * 2
pokemon2['Total']

0       636
1       810
2      1050
3      1250
4       618
       ... 
795    1200
796    1400
797    1200
798    1360
799    1200
Name: Total, Length: 800, dtype: int64

### Sort_values() & value_counts()

1. ***df.sort_values()***
2. ***df.value_counts()***


The ***pandas.sort_values()*** allows us to reorder our dataframe in an ascending or descending order given a column for pandas to work from. This is similar to the excel sort function.

```python
import pandas as pd
df = pd.read_csv('random.csv')
df


df.sort_values(by=['some_column'], ascending = True)
```
In the above code snippet, we are sorting our *random.csv* pandas data frame by the column *some_column* in ascending order. To read more on the ***df.sort_values()*** function, read this [article](https://datatofish.com/sort-pandas-dataframe/).

The second function is ***df.value_counts()***, it allows us to count how many times a specific value/item occurred in the dataframe. This function is best used on a specific column on a data frame, ideally on a column representing categorical data. Categorical data refers to a statistical data type consisting of categorical variables. 

```python
df['column'].value_counts()
```

To read more on some of the advanced functionalities of ***df.value_counts()***, please refer to the pandas documentation or this [article](https://towardsdatascience.com/getting-more-value-from-the-pandas-value-counts-aa17230907a6).

In [60]:
# list Unique Values
pokemon.sort_values(by = ['Attack', "HP"], ascending=[True,False])

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
121,113,Chansey,Normal,NaN,450,250,5,5,35,105,50,1,False
488,440,Happiny,Normal,NaN,220,100,5,5,15,65,30,4,False
261,242,Blissey,Normal,NaN,540,255,10,10,75,135,55,2,False
139,129,Magikarp,Water,NaN,200,20,10,55,15,20,80,1,False
230,213,Shuckle,Bug,Rock,505,20,10,230,10,230,5,2,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
426,384,RayquazaMega Rayquaza,Dragon,Flying,780,105,180,100,180,100,115,3,True
424,383,GroudonPrimal Groudon,Ground,Fire,770,100,180,160,150,90,90,3,True
429,386,DeoxysAttack Forme,Psychic,NaN,600,50,180,20,180,20,150,3,True
232,214,HeracrossMega Heracross,Bug,Fighting,600,80,185,115,40,105,75,2,False


In [64]:
# How many unique Values
pokemon['Type 1'].nunique()

18

### How to Query or Filter Data with Conditions?

- We can extract specific data from our dataframe based on a specific condition. We will be using the syntax below. Pandas will return a subset of the dataframe based on the given condition. 

```python
df[<insert_condition>]
```

Conditions follow the generic boolean logic in Python. Below is a cheat sheet python boolean logic.

**Conditional Logic:** 

Conditional logic refers to the execution of different actions based on whether a certain condition is met. In programming, these conditions are expressed by a set of symbols called **Boolean Operators**. 

| Boolean Comparator | Example | Meaning                         |
|--------------------|---------|---------------------------------|
| >                  | x > y   | x is greater than y             |
| >=                 | x >= y  | x is greater than or equal to y |
| <                  | x < y   | x is less than y                |
| <=                 | x <= y  | x is less than or equal to y    |
| !=                 | x != y  | x is not equal to y             |
| ==                 | x == y  | x is equal to y                 |




In [70]:
# Step 1: Create a filter
the_filter = pokemon["Total"] >= 600
pokemon[the_filter]

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
7,6,CharizardMega Charizard X,Fire,Dragon,634,78,130,111,130,85,100,1,False
8,6,CharizardMega Charizard Y,Fire,Flying,634,78,104,78,159,115,100,1,False
12,9,BlastoiseMega Blastoise,Water,NaN,630,79,103,120,135,115,78,1,False
102,94,GengarMega Gengar,Ghost,Poison,600,60,65,80,170,95,130,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


In [74]:
# Step 2: Apply Filter
pokemon.where(pokemon["Attack"] < 50, "Weak Pokemon")

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon
2,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon
3,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon
4,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon
796,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon
797,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon
798,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon


Alternativly we can use the pandas **.where()** function, which has the following syntax.


```python 
df.where(<condition>, <What to fill inplace where the condition is not True>) # default is Nan
```

### Grouping and Aggregation 

Grouping and aggregation can be used to calculate statistics on groups in the data.

**Common Aggregation Functions**
- mean()
- median()
- sum()
- count()


In [82]:
# simple groupby
pokemon.groupby("Type 1",as_index=False)["Total"].mean()

,Type 1,Total
0,Bug,378.927536
1,Dark,445.741935
2,Dragon,550.531250
3,Electric,443.409091
4,Fairy,413.176471
5,Fighting,416.444444
6,Fire,458.076923
7,Flying,485.000000
8,Ghost,439.562500
9,Grass,421.142857


- By default, `groupby()` assigns the variable that we're grouping on (in this case `Type 1`) to the index of the output data
- If we use the keyword argument `as_index=False`, the grouping variable is instead assigned to a regular column
  - This can be useful in some situations, such as data visualization functions which expect the relevant variables to be in columns rather than the index

In [ ]:
# grouby with as_index set to False


We can use the `agg` method to compute multiple aggregate functions on our data, for example minimum, maximum and mean attack in Type 1

In [84]:
# combine multiple agg functions 
pokemon.groupby("Type 1",as_index=False).agg(['sum',"min","mean"])

C:\Users\juju\AppData\Local\Temp\ipykernel_4740\2033069343.py:2: FutureWarning: ['Name', 'Type 2'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  pokemon.groupby("Type 1",as_index=False).agg(['sum',"min","mean"])


#                   Total                     HP                 \
            sum  min        mean    sum  min        mean   sum min       mean   
Type 1                                                                          
Bug       23080   10  334.492754  26146  194  378.927536  3925   1  56.884058   
Dark      14302  197  461.354839  13818  220  445.741935  2071  35  66.806452   
Dragon    15180  147  474.375000  17617  300  550.531250  2666  41  83.312500   
Electric  15994   25  363.500000  19510  205  443.409091  2631  20  59.795455   
Fairy      7642   35  449.529412   7024  218  413.176471  1260  35  74.117647   
Fighting   9824   56  363.851852  11244  210  416.444444  1886  30  69.851852   
Fire      17025    4  327.403846  23820  250  458.076923  3635  38  69.903846   
Flying     2711  641  677.750000   1940  245  485.000000   283  40  70.750000   
Ghost     15568   92  486.500000  14066  275  439.562500  2062  20  64.437500   
Grass     24141    1  344.871429  29480  180  421.142857  4709  30  67.271429   
Ground    11401   27  356.281250  14000  265  437.500000  2361  10  73.781250   
Ice       10165  124  423.541667  10403  250  433.458333  1728  36  72.000000   
Normal    31279   16  319.173469  39365  190  401.683673  7573  30  77.275510   
Poison     7050   23  251.785714  11176  245  399.142857  1883  35  67.250000   
Psychic   21706   63  380.807018  27129  198  475.947368  4026  20  70.631579   
Rock      17280   74  392.727273  19965  280  453.750000  2876  30  65.363636   
Steel     11957  208  442.851852  13168  300  487.703704  1761  40  65.222222   
Water     33946    7  303.089286  48211  200  430.455357  8071  20  72.062500   

         Attack  ...    Sp. Def Speed                 Generation      \
            sum  ...       mean   sum min        mean        sum min   
Type 1           ...                                                   
Bug        4897  ...  64.797101  4256   5   61.681159        222   1   
Dark       2740  ...  69.516129  2361  20   76.161290        125   2   
Dragon     3588  ...  88.843750  2657  40   83.031250        124   1   
Electric   3040  ...  73.704545  3718  35   84.500000        144   1   
Fairy      1046  ...  84.705882   826  15   48.588235         70   1   
Fighting   2613  ...  64.703704  1784  25   66.074074         91   1   
Fire       4408  ...  72.211538  3871  20   74.442308        167   1   
Flying      315  ...  72.500000   410  55  102.500000         22   5   
Ghost      2361  ...  76.468750  2059  20   64.343750        134   1   
Grass      5125  ...  70.428571  4335  10   61.928571        235   1   
Ground     3064  ...  62.750000  2045  10   63.906250        101   1   
Ice        1746  ...  76.291667  1523  25   63.458333         85   1   
Normal     7200  ...  63.724490  7012   5   71.551020        299   1   
Poison     2091  ...  64.392857  1780  25   63.571429         71   1   
Psychic    4073  ...  86.280702  4645  20   81.491228        193   1   
Rock       4086  ...  75.477273  2460  10   55.909091        152   1   
Steel      2503  ...  80.629630  1492  23   55.259259        104   2   
Water      8305  ...  70.517857  7388  15   65.964286        320   1   

                   Legendary                   
              mean       sum    min      mean  
Type 1                                         
Bug       3.217391         0  False  0.000000  
Dark      4.032258         2  False  0.064516  
Dragon    3.875000        12  False  0.375000  
Electric  3.272727         4  False  0.090909  
Fairy     4.117647         1  False  0.058824  
Fighting  3.370370         0  False  0.000000  
Fire      3.211538         5  False  0.096154  
Flying    5.500000         2  False  0.500000  
Ghost     4.187500         2  False  0.062500  
Grass     3.357143         3  False  0.042857  
Ground    3.156250         4  False  0.125000  
Ice       3.541667         2  False  0.083333  
Normal    3.051020         2  False  0.020408  
Poison    2.535714         0  F

We can also use `agg` to compute different agg functions for different columns:

In [86]:
agg_dict = {
    'Attack' : 'mean',
    'Defense': ['min', 'max']
}

In [88]:
pokemon.groupby(['Type 1', 'Legendary'], as_index = False).agg(agg_dict)

Type 1 Legendary      Attack Defense     
                              mean     min  max
0        Bug     False   70.971014      30  230
1       Dark     False   86.862069      30  125
2       Dark      True  110.500000      90   95
3     Dragon     False  103.400000      35  130
4     Dragon      True  126.666667      80  121
5   Electric     False   66.125000      15  115
6   Electric      True   98.750000      70   85
7      Fairy     False   57.187500      28   95
8      Fairy      True  131.000000      95   95
9   Fighting     False   96.777778      30   95
10      Fire     False   82.191489      37  140
11      Fire      True  109.000000      85  120
12    Flying     False   50.000000      35   80
13    Flying      True  107.500000      70   80
14     Ghost     False   71.366667      30  145
15     Ghost      True  110.000000     100  120
16     Grass     False   72.119403      30  131
17     Grass      True   97.666667      72  100
18    Ground     False   88.000000      25  130
19    Ground      True  150.000000      90  160
20       Ice     False   73.227273      15  184
21       Ice      True   67.500000     100  100
22    Normal     False   72.083333       5  126
23    Normal      True  140.000000     110  120
24    Poison     False   74.678571      35  120
25   Psychic     False   54.953488      15  120
26   Psychic      True  122.142857      20  160
27      Rock     False   89.925000      40  168
28      Rock      True  122.250000      90  200
29     Steel     False   92.086957      50  230
30     Steel      True   96.250000     100  150
31     Water     False   72.777778      20  180
32     Water      True  111.250000      90  115

### Challenge 1 (20 minutes)

Let's play around with Pandas on a more intricate dataset: a dataset on wines!

**Challenge 14 from the 21 Day Data Challenge** 

Dot's neighbour said that he only likes wine from Stellenbosch, Bordeaux, and the Okanagan Valley, and that the sulfates can't be that high. The problem is, Dot can't really afford to spend tons of money on the wine. Dot's conditions for searching for wine are: 
1. Sulfates cannot be higher than 0.6. 
2. The price has to be less than  $20. 

Use the above conditions to filter the data for questions **2 and 3** below. 

**Questions:**
1. Where is Stellenbosch, anyway? How many wines from Stellenbosch are there in the *entire dataset*? 
2. *After filtering with the 2 conditions*, what is the average price of wine from the Bordeaux region? 
3. *After filtering with the 2 conditions*, what is the least expensive wine that's of the highest quality from the Okanagan Valley?



**Stretch Question:**
1. What is the average price of wine from Stellenbosch, according to the entire unfiltered dataset? 


**Note: Check the dataset to see if there are missing values; if there are, fill in missing values with the mean.**


In [118]:
import pandas as pd

df = pd.read_csv("winequality-red_2.csv")
#1.
df['region'].value_counts()

filter_sulphates


La Rjoja            341
Bordeaux            264
Colchagua Valley    260
Okanagan Valley     256
Willamette          233
Marlborough         210
Stellenbosch         35
Name: region, dtype: int64

### Challenge 2 (25 minutes)

**Challenge 21 from the 21DDC (Adapted)**

Dot wants to play retro video games with all their new friends! Help them figure out which games would be best.

Questions: 
    
1. What is the top 5 best selling games released before the year 2000.

     -  **Note**: Use Global_Sales
    
    
2. Create a new column called Aggregate_Score, which returns the proportional average between Critic Score and User_Score based on Critic_Count and User_Count. Plot a horizontal bar chart of the top 5 highest rated games by Aggregate_Score, not published by Nintendo before the year 2000. From this bar chart, what is the highest rated game by Aggregate_Score?

    -  **Note**: Critic_Count should be filled with the mean. User_Count should be filled with the median.
    
    
#### In the exercise above, there is some missing values in the dataset. Look up the pandas documentation to figure out how to fill missing values in a column. You will be using the **fillna()** function.   

In [ ]:
# code here

# HINT

**How to create the Aggregate Score Column?**

\begin{equation*}
AggregateScore = \frac{(CriticCount * CriticScore)+(UserCount * UserScore)}{UserCount + CriticCount}
\end{equation*}

**Check Your Column Values**

The Critic_Score column is scored out of 100. The User_Score column is scored out of 10. You will need to modify one of the columns to match the other.

## Documentation

In the meantime, check out pandas the user guide in the [pandas documentation](https://pandas.pydata.org/docs/user_guide/index.html#user-guide).

-------
**Why should I use the documentation?**

On the job as a data scientist or data analyst, more often than not, you may find yourself looking up the documentation of a particular function or plugin you use. Don't worry if there are a few functions you don't know by heart. However, there are just too many to know! An essential skill is to learn how to navigate documentation and understand how to apply the examples to your work. 

--------

Additional resources:

- To learn more about these topics, as well as other topics not covered here (e.g. reshaping, merging, additional subsetting methods, working with text data, etc.) check out [these introductory tutorials](https://pandas.pydata.org/docs/getting_started/index.html#getting-started) from the `pandas` documentation
- To learn more about subsetting your data, check out [this tutorial](https://pandas.pydata.org/docs/getting_started/intro_tutorials/03_subset_data.html#min-tut-03-subset)
- This [pandas cheatsheet](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf) may also be helpful as a reference.